In [3]:
import pandas as pd
import numpy as np
import nltk
import operator
import string
import spacy

from collections import Counter
from sklearn.neighbors import NearestNeighbors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nlp = spacy.load("en")
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/teemu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/teemu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
pd.set_option('display.max_columns', 500)
PATH ="../data/repositories-1.2.0-2018-03-12.csv"
dataRepo = pd.read_csv(PATH, nrows = 500000, index_col=False)
dataRepo.head()

/Users/teemu/git_projects/codescoop/codescoop-models/notebook-venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (16,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,Host Type,Name with Owner,Description,Fork,Created Timestamp,Updated Timestamp,Last pushed Timestamp,Homepage URL,Size,Stars Count,Language,Issues enabled,Wiki enabled,Pages enabled,Forks Count,Mirror URL,Open Issues Count,Default branch,Watchers Count,UUID,Fork Source Name with Owner,License,Contributors Count,Readme filename,Changelog filename,Contributing guidelines filename,License filename,Code of Conduct filename,Security Threat Model filename,Security Audit filename,Status,Last Synced Timestamp,SourceRank,Display Name,SCM type,Pull requests enabled,Logo URL,Keywords
0,1,GitHub,brianmhunt/knockout-modal,Opinionated modals with Knockout.js,False,2014-09-15 01:21:34 UTC,2016-12-28 16:33:17 UTC,2016-12-18 18:31:32 UTC,http://brianmhunt.github.io/knockout-modal/,512,7,JavaScript,True,True,True,0,NaN,1,master,2,24038237,NaN,MIT,1,README.md,NaN,NaN,LICENSE,NaN,NaN,NaN,NaN,2016-05-27 15:42:48 UTC,5,GitHub,NaN,NaN,NaN,NaN
1,2,GitHub,SteveSanderson/knockout.mapping,Object mapping plugin for KnockoutJS,False,2010-11-01 09:27:43 UTC,2018-02-11 10:04:55 UTC,2017-06-21 22:54:45 UTC,NaN,924,543,JavaScript,True,True,False,782,NaN,85,master,61,1041356,NaN,MIT,21,README.md,NaN,NaN,LICENSE,NaN,NaN,NaN,NaN,2016-05-27 15:21:05 UTC,8,GitHub,NaN,git,NaN,NaN
2,3,GitHub,azman-co/knockout-model,A set of useful model and collection features ...,True,2014-09-13 03:14:07 UTC,2017-03-18 22:40:02 UTC,2015-01-14 02:01:03 UTC,NaN,472,1,JavaScript,False,True,False,0,NaN,0,master,1,23984550,devco/knockup,NaN,5,README.md,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-27 16:07:08 UTC,2,GitHub,NaN,NaN,NaN,NaN
3,4,GitHub,zonuexe/aozora-ruby-parser.js,Aozora-bunko ruby parser,False,2014-12-27 21:02:09 UTC,2016-12-28 16:45:20 UTC,2015-01-07 18:04:42 UTC,http://zonuexe.github.io/aozora-ruby-parser.js/,536,3,JavaScript,True,True,True,1,NaN,2,master,1,28546273,NaN,NaN,1,README.md,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-27 17:22:36 UTC,3,GitHub,NaN,NaN,NaN,NaN
4,5,GitHub,immense/knockout-pickatime,Pickadate timepicker binding for Knockout.js,False,2014-12-04 21:13:48 UTC,2017-03-18 22:40:04 UTC,2014-12-11 16:12:08 UTC,http://rawgit.com/immense/knockout-pickatime/m...,192,1,CoffeeScript,True,True,False,0,NaN,0,master,2,27560378,NaN,MIT,1,README.md,NaN,NaN,LICENSE,NaN,NaN,NaN,NaN,2016-05-27 18:52:59 UTC,4,GitHub,NaN,NaN,NaN,NaN


In [15]:
# build functions that we need for processing descriptions and data

# converting variables to dummies
def getDummies (data, feature):
    dummies = pd.get_dummies(data[feature])
    data = pd.concat([data, dummies], axis=1)
    data = data.drop([feature],axis=1)
    return data

def get_neighbors_improved(instance, data, labels):
    index = labels[labels == instance].index[0]
    inst = np.array(data[index]).reshape(1, -1)
    distances, indices = nbrs.kneighbors(inst)
    # loc OR iloc?
    return labels.iloc[indices[0]], np.round(distances, 3)

# processing the descriptions

dataRepo.Keywords.fillna(0, inplace=True)
dataRepo.Description.fillna(0, inplace=True)
dataRepo = dataRepo[dataRepo.Description != 0]

descriptions = list(dataRepo.Description)

stop_words = set(stopwords.words('english'))

In [16]:
# this takes forever
descr_lemmas = []
for element in descriptions:
    descr = [token.lemma_ for token in nlp(element) if token.is_stop != True and token.is_punct != True]
    descr = " ".join(descr)
    descr_lemmas.append(descr.lower())
dataRepo['Description'] = descr_lemmas

In [9]:
# for small data
topWords = nlp(" ".join(descr_lemmas))
word_freq = Counter(topWords)
topWords = dict(word_freq.most_common(500)).keys()

In [17]:
# for large data
empty = []
word_freq = Counter(empty)

for i in range(44):
    j = i*10000
    j2 = i*10000+10000
    if(j2 > len(descr_lemmas)):
        j2 = len(descr_lemmas)
    words = nlp(" ".join(descr_lemmas[j:j2]))
    word_freq += Counter(words)

topWords = dict(word_freq.most_common(500)).keys()


In [10]:
# Scale numeric variables using logarithms and normalization, this works well

numeric_vars = ['Stars Count', 'Contributors Count', 'Forks Count']
knn_num = dataRepo[numeric_vars]

knn_num = knn_num.apply(lambda col:((np.log(col+0.5)-np.mean(np.log(col+0.5)))/np.std(np.log(col+0.5))), axis=0)

# Check the distributions of numeric variables
'''
import matplotlib.pyplot as plt
import scipy.stats as scs
for var in numeric_vars:
    print(scs.describe(knn_num[var]))
    plt.hist(knn_num[var])
    plt.show()
'''
# add dummies for language and keywords
knn_data = dataRepo[['Language', 'Description']]
knn_data = getDummies(knn_data, 'Language')

knn_data.head()

,Description,ActionScript,ApacheConf,C,CSS,Clojure,CoffeeScript,D,Elixir,Erlang,HTML,Java,JavaScript,Makefile,Nix,Objective-C,Objective-J,PHP,Pascal,PureScript,Python,Ruby,Rust,Scala,Shell,Swift,TeX,TypeScript,XSLT
0,opinionated modal knockout.js,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,object mapping plugin knockoutj,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,set useful model collection feature knockout,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,aozora bunko ruby parser,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,pickadate timepicker bind knockout.js,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
for word in topWords:
    newcol = np.zeros(len(knn_data))
    i = 0;
    for row in knn_data['Description']:
        if(str(word) in row):
            newcol[i] = 0.25
        i += 1
    knn_data[word] = newcol

# join numeric variables to knn data
knn_data[numeric_vars] = knn_num
knn_data = knn_data.drop(['Description'],axis=1)
knn_data.head()

names = dataRepo['Name with Owner']

In [12]:
# DEMO

# Making a NN model, this takes a while but then retrieving info is very fast

knn_data = np.array(knn_data)
nbrs = NearestNeighbors(n_neighbors=11, algorithm='auto').fit(knn_data)

nbrsTEST, dists = get_neighbors_improved("immense/knockout-pickatime", knn_data, names)
nbrsTEST
dists

array([[0.   , 1.053, 1.136, 1.225, 1.275, 1.346, 1.346, 1.415, 1.429,
        1.436, 1.437]])

In [13]:
import pickle
pickle.dump(nbrs, open('../data/knn.pickle', 'wb'))
np.save("../data/knn_data.npy", knn_data)
np.savetxt("../data/knn_labels.csv", names, fmt='%s')
